In [1]:
from pprint import pprint
from datetime import datetime
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
from utils import *
from utils import markdown

def reduce_repeated_elements(arr):
    mask = np.concatenate(([True], np.diff(arr) != 0))
    reduced_arr = arr[mask]
    return reduced_arr

def compute_transition_probabilities(arr, normalized=True):
    num_integers = np.max(arr) + 1
    transition_matrix = np.zeros((num_integers, num_integers), dtype=[int,float][int(normalized)])
    # Create an array with shifted elements
    shifted_arr = np.roll(arr, -1)
    # Count occurrences of transitions
    unique_transitions, counts = np.unique(arr * num_integers + shifted_arr, return_counts=True)
    transition_matrix[unique_transitions // num_integers, unique_transitions % num_integers] = counts
    # Normalize transition matrix
    if normalized:
        transition_matrix /= np.sum(transition_matrix, axis=1, keepdims=True)
    return transition_matrix

def task_transition_matrix(eye_data, use_none=False, normalized=False):
    for i, (task, window_properties) in enumerate(sorted(ALL_WINDOW_PROPERTIES.items()), start=int(use_none)):
        intaskbox = in_box(eye_data['x'].to_numpy(), eye_data['y'].to_numpy(), window_properties['position'], window_properties['size'])
        eye_data.loc[intaskbox, 'task'] = i  
    task_data = reduce_repeated_elements(eye_data['task'].to_numpy())
    task_t_probs = compute_transition_probabilities(task_data, normalized=False)
    labels = (["none"] if use_none else []) + list(sorted(ALL_WINDOW_PROPERTIES.keys()))
    return pd.DataFrame(task_t_probs, index=labels, columns=labels)
    
#given a np array of shape (n,2) that contains intervals of time with an associated label (1-m) and a total duration of D seconds, how would I discretize this into an array with each element representing  elements using 0 to stand for values that were not in any interval, and the associated label to stand for any element that is 

In [66]:

GAZE_ONLY = False 

def merge_intervals_nonoverlapping(intervals, labels=None):
    if labels is None:
        labels = np.arange(len(intervals))
    labels = np.concatenate([np.full((x.shape[0],1),i) for x, i in zip(intervals, labels)])
    intervals = np.concatenate(intervals, axis=0)
    indx = np.argsort(intervals[:,0]) # sort by start times
    intervals, labels = intervals[indx], labels[indx]
    
    return pd.DataFrame(dict(t1 = intervals[:,0], t2=intervals[:,1], task=labels[:,0]))
    
def in_intervals(intervals, points):
    return (points >= intervals[:, 0][:, np.newaxis]) & (points <= intervals[:, 1][:, np.newaxis])

def plot_intervals(intervals, ax, alpha=0.25, color='red'):
    for interval in intervals:
        ax.axvspan(*interval, alpha=alpha, color=color, linewidth=0)

def transition_pattern_count(data, n=2):
    tp = sliding_window_view(data, (n,)).squeeze()
    u, c = np.unique(tp, axis=0, return_counts=True)
    columns = [f"t{i}" for i in range(n)] + ["count"]
    return pd.DataFrame(data=np.concatenate([u,c[:,np.newaxis]], axis=1), columns=columns)

def in_during_still(intervals1, intervals2): # computes a dataframe tells us about the overlap of interval1 and interval2
     # if the start of the gaze is within a failure interval, it was already failed when they looked.
    fstart = in_intervals(intervals1, intervals2['t1'].to_numpy()).any(axis=0)
    # if the start of the failure is within the gaze interval, it started during the gaze.
    fduring = in_intervals(intervals2[['t1','t2']].to_numpy(), intervals1[:,0]).any(axis=1) 
    # if the end of the gaze is in a failure interval, it continues after they look away.
    fend = in_intervals(intervals1, intervals2['t2'].to_numpy()).any(axis=0) 
    # (fstart OR fduring) AND NOT fend will tell us that the failure was resolved during the look
    return pd.DataFrame(dict(fstart=fstart, fduring=fduring, fend=fend))

def compute_failure_intervals(data, task, start_time, finish_time):
    data = data['tasks'][task]
    return merge_intervals([compute_time_intervals(x.failure, x.timestamp, start_time, finish_time).intervals for x in data.values()])

def compute_warning_intervals(data, task, start_time, finish_time):
    data = data['warnings'][task]
    return compute_time_intervals(data['value'].astype(int), data['timestamp'], start_time, finish_time).intervals
    
def compute_eyetracking_intervals(data, task, start_time, finish_time, gaze_only=False):
    data = data['eye_tracking']
    window_properties = ALL_WINDOW_PROPERTIES[task]
    # compute gaze intervals
    intaskbox = in_box(data['x'].to_numpy(), data['y'].to_numpy(), window_properties['position'], window_properties['size'])
    if gaze_only: # USE THIS IF WE ONLY WANT GAZE
        gaze = data['gaze'].to_numpy().astype(bool)
        intaskbox = np.logical_and(intaskbox, gaze) 
    return compute_time_intervals(intaskbox, data['timestamp'].to_numpy(), start_time, finish_time, pad='next').intervals

def get_interval_data(_data2):
    start_time, finish_time =  _data2['start_time'], _data2['finish_time']
    tasks = sorted(_data2['tasks'].keys()) # ordering of tasks so that we can index later, keep them sorted.
    # compute failure intervals
    failure_intervals = [compute_failure_intervals(_data2, task, start_time, finish_time) for task in tasks]
    # compute eyetracking intervals
    eyetracking_intervals = [compute_eyetracking_intervals(_data2, task, start_time, finish_time, gaze_only=GAZE_ONLY) for task in tasks]
    # compute warning intervals
    warning_intervals = [compute_warning_intervals(_data2, task, start_time, finish_time) for task in tasks]
    # combine eyetracking intervals labelling each row with the task
    # | t1 | t2 | task |: tasks that the participant is looking at and when.
    df_eyetracking = merge_intervals_nonoverlapping(eyetracking_intervals)

    # compute failure data, check when failures are happening and where the participant is looking
    # this dataframe will contain bools indicating start,in,during for each row in df_eyetracking
    # | START | IN | DURING |
    df_failure = [in_during_still(fintervals, df_eyetracking) for fintervals in failure_intervals]
    # do the same for warnings.
    df_warning = [in_during_still(wintervals, df_eyetracking) for wintervals in warning_intervals]
    return SimpleNamespace(df_warning=df_warning, df_failure=df_failure, df_eyetracking=df_eyetracking, 
                           failure_intervals=failure_intervals, eyetracking_intervals=eyetracking_intervals,
                           warning_intervals=warning_intervals, tasks=tasks)

def get_counts(df_eyetracking, indx, tasks):
    return df_eyetracking.iloc[indx]['task'].value_counts().reindex(range(len(tasks)), fill_value=0).to_numpy()


def compute_failure_interval_statistics(data, statistic):
    df = pd.DataFrame(columns=['participant', 'experiment', *sorted(ALL_WINDOW_PROPERTIES.keys()), 'total'])
    for participant, _data1 in data.items():
        for experiment, _data2 in _data1.items():
            start_time, finish_time =  _data2['start_time'], _data2['finish_time']
            stats, all_failure_intervals = [], []
            for task, _data3 in sorted(_data2['tasks'].items()):
                failure_intervals = [compute_time_intervals(x.failure, x.timestamp, start_time, finish_time).intervals for x in _data3.values()]
                failure_intervals = merge_intervals(failure_intervals) # merge them to get failures for a single task
                stats.append(statistic(failure_intervals, start_time, finish_time))
                all_failure_intervals.append(failure_intervals)
            total_failure = merge_intervals(all_failure_intervals) # overlapping total
            total_stat = statistic(total_failure, start_time, finish_time)
            df.loc[len(df)] = [participant, experiment[3:], *stats, total_stat]
    return df


def get_scores(failure_statistic, invert=True):
    data = load_nested_dict('data/Processed') 
    df_p = compute_failure_interval_statistics(data, failure_statistic)
    df_p = df_p[['participant', 'experiment', 'total']]
    df_p = df_p.pivot(index='participant', columns='experiment', values='total')
    return df_p
    #exp = df_p.reset_index(drop=True).select_dtypes(include="number").columns.to_list()


pd.set_option("display.max_rows", 1000)

stats = dict()


data = load_nested_dict('data/Processed') 
for participant, _data1 in data.items():
    stats[participant] = dict()
    for experiment, _data2 in _data1.items():
        exp_stats = stats[participant][experiment[3:]] = dict()
        result = get_interval_data(_data2)
        # COMPUTE SOME STATISTICS!
        
        # Q1: how often are participants looking at a task when it fails
        # NOTE: tasks can also fail when the participant is not looking at any task.
        def _gen():
            for task, dff in zip(result.tasks, result.df_failure):
                indx, tindx = np.arange(dff.shape[0])[dff['fduring']], result.tasks.index(task)
                counts = get_counts(result.df_eyetracking, indx, result.tasks)
                yield task, *counts
        looking_at_when_fails = pd.DataFrame(_gen(), columns=['looking_at', *result.tasks])
        exp_stats['looking_at_when_fails'] = looking_at_when_fails
        
        for task, dff in zip(result.tasks, result.df_failure):
            # whether the failure was resolve during a look.
            dff['fresolved']  = np.logical_or(dff['fstart'].to_numpy(), dff['fduring'].to_numpy())
            dff['fresolved']  = np.logical_and(dff['fresolved'], np.logical_not(dff['fend']))
            dff['looking_at'] = result.df_eyetracking['task']
            dff['task'] = result.tasks.index(task)
        
        #print(experiment)
       
        exp_stats['resolved_while_looking'] = pd.DataFrame(columns=['task', 'resolved', 'total', 'resolved/total'])
        for task, dff in zip(result.tasks, result.df_failure):
            # Q2: how often was the task resolved while the participant was looking at it?
            resolved = dff[dff['fresolved']]
            resolved_while_looking = resolved['task'] == resolved['looking_at']
            #print(f"{f'Resolved {task} while looking':<33} : {resolved_while_looking.sum()}/{resolved.shape[0]}")
            exp_stats['resolved_while_looking'].loc[len(exp_stats['resolved_while_looking'])] = [task, resolved_while_looking.sum(), resolved.shape[0], f"{resolved_while_looking.sum()}/{resolved.shape[0]}"]
        
        exp_stats['task_swaps'] = pd.DataFrame(columns=['task', 'swaps'])# 'mean', 'std', 'min', 'max'])
        # Q3: how many task swaps before resolving a failure
        for task, dff in zip(result.tasks, result.df_failure):
            failing = dff[np.logical_or(dff['fresolved'], dff['fend'])]
            indx = np.array([0] + (np.arange(failing.shape[0])[failing['fresolved']] + 1).tolist())
            swaps = np.array((indx[1:] - indx[:-1]) - 1) # counting transitions not intervals
            exp_stats['task_swaps'].loc[len(exp_stats['task_swaps'])] = [task, swaps]
            #print(f"Number of task swaps for each failure: {swaps}")
            
        #for task, dfw in zip(result.tasks, result.df_warning):
        #    if dfw[['fstart', 'fduring', 'fend']].any().any(): # there was some kind of warning
        #        dfw['looking_at'] = result.df_eyetracking['task']
        #        for t, dff in [(t, (dff['fstart'] | dff['fduring']).to_numpy()) for dff,t in zip(result.df_failure, result.tasks)]:
        #            dfw[f"f{t}"] = dff
                #print(task)
                #print(dfw)
            
        tr2 = transition_pattern_count(result.df_eyetracking['task'], 2)
        tr2[['t0', 't1']] = np.array(result.tasks)[tr2[['t0', 't1']].to_numpy()]
        tr3 = transition_pattern_count(result.df_eyetracking['task'], 3)
        tr3[['t0', 't1', 't2']] = np.array(result.tasks)[tr3[['t0', 't1', 't2']].to_numpy()]
        exp_stats['tr3'] = tr3
        exp_stats['tr2'] = tr2
        
        print(participant)
        markdown(f"##### 2-gram transitions\n {_data[experiment[3:]]['tr2'].to_markdown()}")
        markdown(f"##### 3-gram transitions\n {_data[experiment[3:]]['tr3'].to_markdown()}")   
    
        

P01


##### 2-gram transitions
 |    | t0       | t1       |   count |
|---:|:---------|:---------|--------:|
|  0 | fuel     | fuel     |       4 |
|  1 | fuel     | system   |      18 |
|  2 | fuel     | tracking |      49 |
|  3 | system   | fuel     |      15 |
|  4 | system   | system   |       2 |
|  5 | system   | tracking |      54 |
|  6 | tracking | fuel     |      53 |
|  7 | tracking | system   |      50 |
|  8 | tracking | tracking |       9 |

##### 3-gram transitions
 |    | t0       | t1       | t2       |   count |
|---:|:---------|:---------|:---------|--------:|
|  0 | fuel     | fuel     | system   |       1 |
|  1 | fuel     | fuel     | tracking |       3 |
|  2 | fuel     | system   | fuel     |       6 |
|  3 | fuel     | system   | system   |       1 |
|  4 | fuel     | system   | tracking |      11 |
|  5 | fuel     | tracking | fuel     |      17 |
|  6 | fuel     | tracking | system   |      28 |
|  7 | fuel     | tracking | tracking |       4 |
|  8 | system   | fuel     | system   |       6 |
|  9 | system   | fuel     | tracking |       9 |
| 10 | system   | system   | tracking |       2 |
| 11 | system   | tracking | fuel     |      29 |
| 12 | system   | tracking | system   |      20 |
| 13 | system   | tracking | tracking |       5 |
| 14 | tracking | fuel     | fuel     |       4 |
| 15 | tracking | fuel     | system   |      11 |
| 16 | tracking | fuel     | tracking |      37 |
| 17 | tracking | system   | fuel     |       9 |
| 18 | tracking | system   | system   |       1 |
| 19 | tracking | system   | tracking |      40 |
| 20 | tracking | tracking | fuel     |       7 |
| 21 | tracking | tracking | system   |       2 |

P01


##### 2-gram transitions
 |    | t0       | t1       |   count |
|---:|:---------|:---------|--------:|
|  0 | fuel     | fuel     |       1 |
|  1 | fuel     | system   |      30 |
|  2 | fuel     | tracking |      30 |
|  3 | system   | fuel     |      23 |
|  4 | system   | system   |       2 |
|  5 | system   | tracking |      56 |
|  6 | tracking | fuel     |      36 |
|  7 | tracking | system   |      50 |
|  8 | tracking | tracking |       3 |

...

##### 2-gram transitions
 |    | t0       | t1       |   count |
|---:|:---------|:---------|--------:|
|  0 | fuel     | fuel     |       9 |
|  1 | fuel     | system   |      23 |
|  2 | fuel     | tracking |      47 |
|  3 | system   | fuel     |      26 |
|  4 | system   | system   |       1 |
|  5 | system   | tracking |      52 |
|  6 | tracking | fuel     |      45 |
|  7 | tracking | system   |      55 |
|  8 | tracking | tracking |       8 |

##### 3-gram transitions
 |    | t0       | t1       | t2       |   count |
|---:|:---------|:---------|:---------|--------:|
|  0 | fuel     | fuel     | fuel     |       1 |
|  1 | fuel     | fuel     | system   |       2 |
|  2 | fuel     | fuel     | tracking |       6 |
|  3 | fuel     | system   | fuel     |       9 |
|  4 | fuel     | system   | tracking |      14 |
|  5 | fuel     | tracking | fuel     |      14 |
|  6 | fuel     | tracking | system   |      29 |
|  7 | fuel     | tracking | tracking |       4 |
|  8 | system   | fuel     | fuel     |       4 |
|  9 | system   | fuel     | system   |       7 |
| 10 | system   | fuel     | tracking |      15 |
| 11 | system   | system   | fuel     |       1 |
| 12 | system   | tracking | fuel     |      25 |
| 13 | system   | tracking | system   |      23 |
| 14 | system   | tracking | tracking |       4 |
| 15 | tracking | fuel     | fuel     |       4 |
| 16 | tracking | fuel     | system   |      14 |
| 17 | tracking | fuel     | tracking |      26 |
| 18 | tracking | system   | fuel     |      16 |
| 19 | tracking | system   | system   |       1 |
| 20 | tracking | system   | tracking |      38 |
| 21 | tracking | tracking | fuel     |       5 |
| 22 | tracking | tracking | system   |       3 |

P25


##### 2-gram transitions
 |    | t0       | t1       |   count |
|---:|:---------|:---------|--------:|
|  0 | fuel     | fuel     |       2 |
|  1 | fuel     | system   |      39 |
|  2 | fuel     | tracking |      32 |
|  3 | system   | fuel     |      26 |
|  4 | system   | tracking |      41 |
|  5 | tracking | fuel     |      46 |
|  6 | tracking | system   |      27 |
|  7 | tracking | tracking |       4 |

##### 3-gram transitions
 |    | t0       | t1       | t2       |   count |
|---:|:---------|:---------|:---------|--------:|
|  0 | fuel     | fuel     | system   |       2 |
|  1 | fuel     | system   | fuel     |      20 |
|  2 | fuel     | system   | tracking |      19 |
|  3 | fuel     | tracking | fuel     |      15 |
|  4 | fuel     | tracking | system   |      14 |
|  5 | fuel     | tracking | tracking |       3 |
|  6 | system   | fuel     | fuel     |       1 |
|  7 | system   | fuel     | system   |      11 |
|  8 | system   | fuel     | tracking |      14 |
|  9 | system   | tracking | fuel     |      27 |
| 10 | system   | tracking | system   |      13 |
| 11 | system   | tracking | tracking |       1 |
| 12 | tracking | fuel     | fuel     |       1 |
| 13 | tracking | fuel     | system   |      26 |
| 14 | tracking | fuel     | tracking |      18 |
| 15 | tracking | system   | fuel     |       6 |
| 16 | tracking | system   | tracking |      21 |
| 17 | tracking | tracking | fuel     |       4 |

In [57]:
score_fp = get_scores(Statistics.compute_failure_proportion)    # low = better
score_fp['total'] = score_fp.select_dtypes(include="float").sum(axis=1)
score_fp = score_fp.sort_values('total').reset_index() # lower scores are better
print(score_fp)

experiment participant      icuA      icuB     icuaA     icuaB     total
0                  P01  0.170048  0.459953  0.140658  0.346320  1.116980
1                  P18  0.126635  0.560562  0.083707  0.481855  1.252759
2                  P23  0.129862  0.791822  0.098092  0.728503  1.748279
3                  P10  0.177597  0.747084  0.165719  0.763058  1.853458
4                  P17  0.287987  0.623271  0.175265  0.795597  1.882121
5                  P07  0.191217  0.757468  0.131267  0.846352  1.926304
6                  P08  0.537195  0.753385  0.169511  0.575844  2.035934
7                  P09  0.220562  0.626996  0.490798  0.720709  2.059065
8                  P03  0.740864  0.294093  0.416329  0.771296  2.222582
9                  P06  0.112680  0.793016  0.508977  0.819412  2.234086
10                 P04  0.348942  0.812973  0.346533  0.862713  2.371162
11                 P05  0.635587  0.911663  0.193593  0.863283  2.604126
12                 P25  0.474999  0.709436  0.76146

dict_keys(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P16', 'P17', 'P18', 'P23', 'P24', 'P25'])


In [62]:
scores = score_fp
scores = pd.DataFrame([score_fp.iloc[0], score_fp.iloc[len(score_fp)-1]])
print(scores)

for participant in scores['participant']:
    _data = stats[participant]
    experiments = list(sorted(_data.keys(), key=lambda k:str(reversed(k))))
    markdown(f"## {participant}")
    markdown(f"#### Q1: How often did a task fail while the participant was looking at it?")
    tables = [_data[exp]['looking_at_when_fails'].to_markdown() for exp in experiments]
    embed_markdown_tables(tables, ncols=len(experiments), headers=experiments)
    markdown("NOTE: tasks can also fail when the participant is not looking at any task, this is not recorded here. TODO should it be?")
      
    markdown(f"#### Q2: How often was a task resolved while the participant was looking at it?")
    tables = [_data[exp]['resolved_while_looking'][['task', 'resolved/total']].to_markdown() for exp in experiments]
    embed_markdown_tables(tables, ncols=len(experiments), headers=experiments)

    markdown(f"#### Q3: how many task swaps before resolving a failure?")
    tables = [_data[exp]['task_swaps'].to_markdown() for exp in experiments]
    embed_markdown_tables(tables, ncols=len(experiments), headers=experiments)
    swaps.mean(), swaps.std(), swaps.min() if swaps.shape[0] > 0 else float('nan'), swaps.max() if swaps.shape[0] > 0 else float('nan')
    
    
 
    

experiment participant      icuA      icuB     icuaA     icuaB     total
0                  P01  0.170048  0.459953  0.140658  0.346320  1.116980
15                 P24  0.766149  0.888517  0.810646  0.867995  3.333307


## P01

#### Q1: How often did a task fail while the participant was looking at it?

NOTE: tasks can also fail when the participant is not looking at any task, this is not recorded here. TODO should it be?

#### Q2: How often was a task resolved while the participant was looking at it?

#### Q3: how many task swaps before resolving a failure?

/home/ben/anaconda3/envs/icua/lib/python3.9/site-packages/tabulate/__init__.py:108: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  or (len(row) >= 2 and row[1] == SEPARATING_LINE)


NameError: name 'exp' is not defined

In [4]:
        
        # Q: Transition probabilities
        #t_probs = compute_transition_probabilities(df_eyetracking['task'].to_numpy(), normalized=False)
        #markdown("### Task Transition Tables")
        #markdown(pd.DataFrame(t_probs, index=tasks, columns=tasks).to_markdown())
        tr2 = transition_pattern_count(result.df_eyetracking['task'], 2)
        tr2[['t0', 't1']] = np.array(result.tasks)[tr2[['t0', 't1']].to_numpy()]
        #markdown(f"##### 2-gram transitions\n {tr2.to_markdown()}")
        tr3 = transition_pattern_count(result.df_eyetracking['task'], 3)
        tr3[['t0', 't1', 't2']] = np.array(result.tasks)[tr3[['t0', 't1', 't2']].to_numpy()]
        #markdown(f"##### 3-gram transitions\n {tr3.to_markdown()}")    
        # NOTE: participants transition to and from tracking a lot more.
        # They also seem to do loops of the three tasks, since tracking is kind of on the way, it will be seen more
        # TODO do all participants follow this pattern? probably... its quite efficient!
        
    break
    
# TODO compare mean interval length with and without agents... maybe it improves???



# NOTE: perhaps the agents can improve performance by guiding them to tasks that are at risk of failure...? 
# this might be a bit aggressive... 

IndentationError: unexpected indent (2165638889.py, line 32)

In [ ]:
# COMPUTE TRANSITION PROBABILITIES FOR EYES 
# NOTE: this will give a different result to the above because it only uses gaze data and removes self transitions.
data = load_nested_dict('data/Processed') 
for participant, _data1 in data.items(): #itertools.islice(data.items(), 1):
    for experiment, _data2 in _data1.items():
        start_time, finish_time =  _data2['start_time'], _data2['finish_time']
        eye_data = _data2['eye_tracking']  
        gaze = eye_data['gaze'].to_numpy().astype(bool)
        eye_data['task'] = np.zeros_like(eye_data['x'].to_numpy(), dtype=int)
        eye_data = eye_data.loc[gaze] # take only gaze data.                 
        t_probs = task_transition_matrix(eye_data, use_none=False, normalized=False)
        print(t_probs)
        #gaze = eye_data['gaze'].to_numpy()
    break

        